# Pytorch使用基础

本文主要参考pytorch中文文档。基本思路是：

- 首先简单认识pytorch的基本情况；
- 其次，简单介绍神经网络，并给出torch的入门级实现；
- 接下来，对pytorch的主要构成一一解析；
- 最后是lstm等神经网络结构的实现和一些训练细节。

## torch快速入门


In [ ]:
from __future__ import print_function
import torch
# 构建一个 5x3 的矩阵, 未初始化的:
x = torch.Tensor(5, 3)
print(x)
# 构建一个随机初始化的矩阵:
x = torch.rand(5, 3)
print(x)
# 获得 size:
print(x.size())
# 加法
y = torch.rand(5, 3)
print(x + y)

print(torch.add(x, y))

result = torch.Tensor(5, 3)
torch.add(x, y, out = result)
print(result)

# 可以用类似Numpy的索引来处理所有的张量！
print(x[:, 1])

# 改变大小: 如果你想要去改变tensor的大小, 可以使用 torch.view:
x = torch.randn(4, 4)
y = x.view(16)
z = x.view(-1, 8)  # the size -1 is inferred from other dimensions
print(x.size(), y.size(), z.size())

# 转换一个 Torch Tensor 为 NumPy 数组
a = torch.ones(5)
print(a)

b = a.numpy()
print(b)
# 查看 numpy 数组是如何改变的.a和b是绑定的
a.add_(1)
print(a)
print(b)

# 看改变 np 数组之后 Torch Tensor 是如何自动改变的
import numpy as np
a = np.ones(5)
b = torch.from_numpy(a)
np.add(a, 1, out = a)
print(a)
print(b)

# 可以使用 .cuda 方法将 Tensors 在GPU上运行.
# 只要在  CUDA 是可用的情况下, 我们可以运行这段代码
if torch.cuda.is_available():
    b = b.cuda()
    print(b + b)


import torch
from torch.autograd import Variable

# 创建 variable（变量）:

x = Variable(torch.ones(2, 2), requires_grad = True)
print(x)

# variable（变量）的操作:

y = x + 2
print(y)

# y 由操作创建,所以它有 grad_fn 属性.

print(y.grad_fn)

# y 的更多操作

z = y * y * 3
out = z.mean()

print(z, out)

# 梯度 我们现在开始了解反向传播, out.backward() 与 out.backward(torch.Tensor([1.0])) 这样的方式一样

out.backward()

# 但因 d(out)/dx 的梯度
# 你应该得到一个 4.5 的矩阵. 让我们推导出 out Variable “o”. 我们有 o=1/4∑izi, zi=3(xi+2)^2 和 zi∣∣xi=1=27. 因此, ∂o/∂xi=32(xi+2), 所以 ∂o∂xi∣∣xi=1=9/2=4.5.
print(x.grad)

# 你可以使用自动求导来做很多有趣的事情
x = torch.randn(3)
# torch.norm(input, p=2) → float
#
# 返回输入张量input 的p 范数。
#
# 参数：
#     input (Tensor) – 输入张量
#     p (float,optional) – 范数计算中的幂指数值
x = Variable(x, requires_grad = True)

y = x * 2
print(y.data)
print(y.data.norm())

while y.data.norm() < 1000:
    y = y * 2

print(y)

gradients = torch.FloatTensor([0.1, 1.0, 0.0001])
y.backward(gradients)

print(x.grad)

# 上面整个用计算图的思路去理解就很简单了

""" 神经网络可以使用 torch.nn 包构建.
 
 autograd 实现了反向传播功能, 但是直接用来写深度学习的代码在很多情况下还是稍显复杂,
  torch.nn 是专门为神经网络设计的模块化接口. nn 构建于 Autograd 之上, 
 可用来定义和运行神经网络. nn.Module 是 nn 中最重要的类, 
 可把它看成是一个网络的封装, 包含网络各层定义以及 forward 方法, 
调用 forward(input) 方法, 可返回前向传播的结果.

一个典型的神经网络训练过程如下:

    定义具有一些可学习参数(或权重)的神经网络
    迭代输入数据集
    通过网络处理输入
    计算损失(输出的预测值与实际值之间的距离)
    将梯度传播回网络
    更新网络的权重, 通常使用一个简单的更新规则: weight = weight - learning_rate * gradient

"""
# 定义一个网络:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 卷积层 '1'表示输入图片为单通道, '6'表示输出通道数, '5'表示卷积核为5*5
        # 核心
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # 仿射层/全连接层: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        #在由多个输入平面组成的输入信号上应用2D最大池化.
        # (2, 2) 代表的是池化操作的步幅
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # 如果大小是正方形, 则只能指定一个数字
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # 除批量维度外的所有维度
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)



## 神经网络基础


In [ ]:
# 跟着例子学习PyTorch

# 先使用 numpy 实现网络。
# Numpy 提供了一个n维的数组对象, 并提供了许多操纵这个数组对象的函数.
#  Numpy 是科学计算的通用框架; Numpy 数组没有计算图, 也没有深度学习, 也没有梯度下降等方法实现的接口.
# 但是我们仍然可以很容易地使用 numpy 生成随机数据 并将产生的数据传入双层的神经网络,
#  并使用 numpy 来实现这个网络的正向传播和反向传播:

# -*- coding: utf-8 -*-
import numpy as np

import torch

import torch.nn.functional as F
from torch.autograd import Variable
import matplotlib.pyplot as plt

# N 是一个batch的样本数量; D_in是输入维度;
# H 是隐藏层向量的维度; D_out是输出维度.
N, D_in, H, D_out = 64, 1000, 100, 10

# 创建随机的输入输出数据
x = np.random.randn(N, D_in)
y = np.random.randn(N, D_out)

# 随机初始化权重参数
w1 = np.random.randn(D_in, H)
w2 = np.random.randn(H, D_out)

learning_rate = 1e-6
for t in range(500):
    # 前向计算, 算出y的预测值
    h = x.dot(w1)
    h_relu = np.maximum(h, 0)
    y_pred = h_relu.dot(w2)

    # 计算并打印误差值
    loss = np.square(y_pred - y).sum()
    print(t, loss)

    # 在反向传播中, 计算出误差关于w1和w2的导数
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.T.dot(grad_y_pred)
    grad_h_relu = grad_y_pred.dot(w2.T)
    grad_h = grad_h_relu.copy()
    grad_h[h < 0] = 0
    grad_w1 = x.T.dot(grad_h)

    # 更新权重
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

# Numpy 是一个伟大的框架, 但它不能利用 GPU 加速它数值计算.
#  对于现代的深度神经网络, GPU 往往是提供 50倍或更大的加速,
# 所以不幸的是, numpy 不足以满足现在深度学习的需求.
# 介绍一下最基本的 PyTorch 概念: Tensor .
# PyTorch Tensor 在概念上与 numpy 数组相同:
# Tensor 是一个n维数组, PyTorch 也提供了很多能在这些 Tensor 上操作的函数.
#  像 numpy 数组一样, PyTorch Tensor 也和numpy的数组对象一样不了解深度学习,计算图和梯度下降；
# 它们只是科学计算的通用工具.
# 然而不像 numpy, PyTorch Tensor 可以利用 GPU 加速他们的数字计算.
#  要在 GPU 上运行 PyTorch 张量, 只需将其转换为新的数据类型.

# 将 PyTorch Tensor 生成的随机数据传入双层的神经网络. 就像上面的 numpy 例子一样,
# 我们需要手动实现网络的正向传播和反向传播:
# -*- coding: utf-8 -*-


dtype = torch.FloatTensor
dtype = torch.cuda.FloatTensor  # 取消注释以在GPU上运行

# N 批量大小; D_in是输入尺寸;
# H是隐藏尺寸; D_out是输出尺寸.
N, D_in, H, D_out = 64, 1000, 100, 10

# 创建随机输入和输出数据
x = torch.randn(N, D_in).type(dtype)
y = torch.randn(N, D_out).type(dtype)

# 随机初始化权重
w1 = torch.randn(D_in, H).type(dtype)
w2 = torch.randn(H, D_out).type(dtype)

learning_rate = 1e-6
for t in range(500):
    # 正向传递：计算预测y
    h = x.mm(w1)
    h_relu = h.clamp(min=0)
    y_pred = h_relu.mm(w2)

    # 计算并打印loss
    loss = (y_pred - y).pow(2).sum()
    print(t, loss)

    # 反向传播计算关于损失的w1和w2的梯度
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    grad_w1 = x.t().mm(grad_h)

    # 使用梯度下降更新权重
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

# torch和numpy
np_data = np.arange(6).reshape((2, 3))
torch_data = torch.from_numpy(np_data)
tensor2array = torch_data.numpy()
print(
    '\nnumpy', np_data,
    '\ntorch tensor', torch_data,
    '\ntensor to array', tensor2array
)

# 进行计算的时候 numpy和torch的方式也有所不同
data = np.array([[1, 2], [3, 4]])
tensor = torch.FloatTensor(data)

print(
    '\nnumpy:', data.dot(data),
    '\ntorch:', torch.mm(tensor, tensor)
)

# variable变量
variable = Variable(tensor, requires_grad=True)

t_out = torch.mean(tensor * tensor)
v_out = torch.mean(variable * variable)

print(t_out)
print(v_out)

v_out.backward()
# v_out=1/4*sum(variable*variable) d(v_out)/d(variable)=variable/2
print(variable.grad)

print(variable)

print(variable.data)

print(variable.data.numpy())

# fake data
x = torch.linspace(-5, 5, 200)  # x data (tensor) ,shape=(100,1)
x = Variable(x)
x_np = x.data.numpy()  # 为了画图，转换为numpy的数据形式

y_relu = F.relu(x).data.numpy()
y_sigmoid = F.sigmoid(x).data.numpy()
y_tanh = F.tanh(x).data.numpy()
y_softplus = F.softplus(x).data.numpy()

# plt.figure(1, figsize=(8, 6))
# plt.subplot(221)
# plt.plot(x_np, y_relu, c='red', label='relu')
# plt.ylim((-1, 5))
# plt.legend(loc='best')
#
# plt.subplot(222)
# plt.plot(x_np, y_sigmoid, c='red', label='sigmoid')
# plt.ylim((-0.2, 1.2))
# plt.legend(loc='best')
#
# plt.subplot(223)
# plt.plot(x_np, y_tanh, c='red', label='tanh')
# plt.ylim((-1.2, 1.2))
# plt.legend(loc='best')
#
# plt.subplot(224)
# plt.plot(x_np, y_softplus, c='red', label='softplus')
# plt.ylim((-0.2, 6))
# plt.legend(loc='best')

# plt.show()

# 关系拟合
# unsqueeze见文档（不知道什么用法的都去查文档，文档很清楚）
x = torch.unsqueeze(torch.linspace(-1, 1, 100), dim=1)  # x data (tensor),shape=(100,1)
y = x.pow(2) + 0.2 * torch.rand(x.size())  # noisy y data (tensor), shape=(100, 1)
print(x)
print(y)
# 用 Variable 来修饰这些数据 tensor
x, y = Variable(x), Variable(y)


# 画图
# plt.scatter(x.data.numpy(), y.data.numpy())
# plt.show()

# 建立神经网络
class Net(torch.nn.Module):  # 继承 torch 的 Module
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()  # 继承 __init__ 功能
        # 定义每层用什么样的形式
        self.hidden = torch.nn.Linear(n_feature, n_hidden)  # 隐藏层线性输出
        self.predict = torch.nn.Linear(n_hidden, n_output)  # 输出层线性输出

    def forward(self, x):  # 这同时也是 Module 中的 forward 功能
        # 正向传播输入值, 神经网络分析出输出值
        x = F.relu(self.hidden(x))  # 激励函数(隐藏层的线性值)
        x = self.predict(x)  # 输出值
        return x


net = Net(1, 10, 1)
print(net)  # net 的结构

# 训练神经网络，optimizer 是训练的工具
optimizer = torch.optim.SGD(net.parameters(), lr=0.5)  # 传入 net 的所有参数, 学习率
loss_func = torch.nn.MSELoss()  # 预测值和真实值的误差计算公式 (均方差)

plt.ion()  # 画图
plt.show()
for t in range(100):
    prediction = net(x)  # 喂给 net 训练数据 x, 输出预测值

    loss = loss_func(prediction, y)  # 计算两者的误差

    optimizer.zero_grad()  # 清空上一步的残余更新参数值
    loss.backward()  # 误差反向传播, 计算参数更新值
    optimizer.step()  # 将参数更新值施加到 net 的 parameters 上
    # 接着上面来
    if t % 5 == 0:
        # plot and show learning process
        plt.cla()
        plt.scatter(x.data.numpy(), y.data.numpy())
        plt.plot(x.data.numpy(), prediction.data.numpy(), 'r-', lw=5)
        plt.text(0.5, 0, 'Loss=%.4f' % loss.data[0], fontdict={'size': 20, 'color': 'red'})
        plt.pause(0.1)


## torch.nn


In [ ]:
# 利用pytorch写一个神经网络的示例——来自课程《Mordern deep learning in Python》
import matplotlib.pyplot as plt
from torch.autograd import Variable
from torch import optim
from util import get_normalized_data
import numpy as np
import pandas as pd
import torch

# 第一步是load data，这里采用的是deep learning里的hello world，即识别手写数字的数据集，来自kaggle。
# get the data,
# no need to split now, the fit() function will do it
Xtrain, Xtest, Ytrain, Ytest = get_normalized_data()

# get shapes
_, D = Xtrain.shape
K = len(set(Ytrain))

# 第二步是构建网络结构，使用pytorch构建神经网络要比直接通过numpy手写简单很多。和Keras类似，可以先构建一个sequential，然后逐层添加网络结构即可。
model = torch.nn.Sequential()
# 逐层添加网络即可，在pytorch中是采用add_module()函数。第一个参数是当前层的命名，可以取任何想要的名称。第二个参数就是该层。层要么是linear transformation，要么是activation
# 比如第一层，Linear,参数D是输入的神经元个数，第二个参数500是输出的神经元个数
model.add_module("dense1", torch.nn.Linear(D, 500))
model.add_module("relu1", torch.nn.ReLU())
model.add_module("dense2", torch.nn.Linear(500, 300))
model.add_module("relu2", torch.nn.ReLU())
model.add_module("dense3", torch.nn.Linear(300, K))
# 第三步是构建loss函数，loss函数详情可参考http://pytorch.org/docs/master/nn.html#loss-functions
loss = torch.nn.CrossEntropyLoss(size_average=True)
# 第四步是设置优化函数，Adam是一种常用的优化算法，是一种改良的GD算法。算法需要神经网络的parameters作为参数。
optimizer = optim.Adam(model.parameters())


# 第五步是定义训练过程和预测过程，这部分也是相对最难掌握的一部分。这部分相对TensorFlow和Theano都会麻烦一些。


def train(model, loss, optimizer, inputs, labels):
    """训练过程主要包括：包装输入输出到Variable变量，初始化优化函数，前向传播，反向传播，以及参数更新，详情见每步解释"""
    # 为什么要包装变量到Variable：把Tensor包装到Variable中，它就会开始保存所有计算历史。因此每次运算都会稍微多一些cost；另一方面，在训练循环外部对Variable进行计算操作相对容易。不包装也是可以计算的，并且后面的pytorch版本有柯南高就不需要Variable的这一步了
    inputs = Variable(inputs, requires_grad=False)
    labels = Variable(labels, requires_grad=False)

    # pytorch的梯度计算是累计的，这对有些神经网络是比较好的，因此这里初始化为0
    optimizer.zero_grad()

    # 直接调用model的前向函数即可得到输出
    logits = model.forward(inputs)

    # 然后计算loss
    output = loss.forward(logits, labels)

    # 接着就是进行反向传播计算
    output.backward()

    # 最后是更新参数
    optimizer.step()

    return output.item()


def predict(model, inputs):
    inputs = Variable(inputs, requires_grad=False)
    logits = model.forward(inputs)
    # argmax函数是给出axis维上数组中最大数的索引
    return logits.data.numpy().argmax(axis=1)


# 第六步定义各类超参数并开始训练过程
# 首先是将numpy变量都设置为torch中的张量，注意要指定数据类型
Xtrain = torch.from_numpy(Xtrain).float()
Ytrain = torch.from_numpy(Ytrain).long()
Xtest = torch.from_numpy(Xtest).float()

epochs = 15
batch_size = 32  # 每个batch的大小
n_batches = Xtrain.size()[0] // batch_size  # batch的个数

costs = []
test_accuracies = []
for i in range(epochs):
    cost = 0.
    for j in range(n_batches):
        Xbatch = Xtrain[j * batch_size:(j + 1) * batch_size]
        Ybatch = Ytrain[j * batch_size:(j + 1) * batch_size]
        cost += train(model, loss, optimizer, Xbatch, Ybatch)

    Ypred = predict(model, Xtest)
    acc = np.mean(Ytest == Ypred)
    print("Epoch: %d, cost: %f, acc: %.2f" % (i, cost / n_batches, acc))

    costs.append(cost / n_batches)
    test_accuracies.append(acc)

# 第七步是将训练过程可视化,# EXERCISE: plot test cost + training accuracy too。一般都是把cost和accuracy都可视化出来
# plot the results
plt.plot(costs)
plt.title('Training cost')
plt.show()

plt.plot(test_accuracies)
plt.title('Test accuracies')
plt.show()


## LSTM in Torch



In [ ]:
"""一个简单的pytorch的lstm模型的示例"""
# Author: Robert Guthrie
# 作者：Robert Guthrie

import torch
import torch.autograd as autograd  # torch中自动计算梯度模块
import torch.nn as nn             # 神经网络模块
import torch.nn.functional as F   # 神经网络模块中的常用功能
import torch.optim as optim       # 模型优化器模块

torch.manual_seed(1)

# 第一句代码生成了一个LSTM对象，LSTM类继承自RNNBase类，RNNBase类继承自Module类，
# Module类是pytorch中完成一定网络功能的基类，可以通过继承该类定义自己的神经网络。
# 自己实现神经网络时，一般要重写其forward方法。
# Module实现了__call__方法，这意味着其可被当做可调用方法使用。比如下面有用到lstm()。
# RNNBase的__init__和__forward方法都是要读一读的，理解这个及基本理解了循环网络的机制。
# lstm单元输入和输出维度都是3
lstm = nn.LSTM(3, 3)
# 生成一个长度为5，每一个元素为1*3的序列作为输入，这里的数字3对应于上句中第一个3
inputs = [autograd.Variable(torch.randn((1, 3))) for _ in range(5)]

# 设置隐藏层维度，初始化隐藏层的数据。hidden变量是一个元组，其第一个元素是LSTM隐藏层输出，另一个元素维护隐藏层的状态。
# torch.rand(1,1,3)就是生成了一个维度为(1,1,3)的以一定高斯分布生成的张量
hidden = (autograd.Variable(torch.randn(1, 1, 3)),
          autograd.Variable(torch.randn((1, 1, 3))))

for i in inputs:
    # Step through the sequence one element at a time.
    # after each step, hidden contains the hidden state.
    out, hidden = lstm(i.view(1, 1, -1), hidden)
    print(out)
    print(hidden)


## 神经网络训练技巧


In [ ]:
"""用pytorch写神经网络时，进行dropout的示例。
有dropout的神经网络的区别：
一个是在神经网络结构中增加dropout层；
另一个，注意dropout有两个mode，一是对train data的，一个是对test data的。在train data中进行dropout，但是在test data中是不用的"""

# https://deeplearningcourses.com/c/data-science-deep-learning-in-theano-tensorflow
# https://www.udemy.com/data-science-deep-learning-in-theano-tensorflow
from __future__ import print_function, division
from builtins import range
# Note: you may need to update your version of future
# sudo pip install -U future

# Note: is helpful to look at keras_example.py first


import numpy as np
import matplotlib.pyplot as plt
from util import get_normalized_data

import torch
from torch.autograd import Variable
from torch import optim

# get the data, same as Theano + Tensorflow examples
# no need to split now, the fit() function will do it
Xtrain, Xtest, Ytrain, Ytest = get_normalized_data()

# get shapes
_, D = Xtrain.shape
K = len(set(Ytrain))

# Note: no need to convert Y to indicator matrix


# the model will be a sequence of layers
model = torch.nn.Sequential()

# ANN with layers [784] -> [500] -> [300] -> [10]
# NOTE: the "p" is p_drop, not p_keep
model.add_module("dropout1", torch.nn.Dropout(p=0.2))
model.add_module("dense1", torch.nn.Linear(D, 500))
model.add_module("relu1", torch.nn.ReLU())
model.add_module("dropout2", torch.nn.Dropout(p=0.5))
model.add_module("dense2", torch.nn.Linear(500, 300))
model.add_module("relu2", torch.nn.ReLU())
model.add_module("dropout3", torch.nn.Dropout(p=0.5))
model.add_module("dense3", torch.nn.Linear(300, K))
# Note: no final softmax!
# just like Tensorflow, it's included in cross-entropy function


# define a loss function
# other loss functions can be found here:
# http://pytorch.org/docs/master/nn.html#loss-functions
loss = torch.nn.CrossEntropyLoss(size_average=True)
# Note: this returns a function!
# e.g. use it like: loss(logits, labels)


# define an optimizer
# other optimizers can be found here:
# http://pytorch.org/docs/master/optim.html
optimizer = optim.Adam(model.parameters(), lr=1e-4)


# define the training procedure
# i.e. one step of gradient descent
# there are lots of steps
# so we encapsulate it in a function
# Note: inputs and labels are torch tensors
def train(model, loss, optimizer, inputs, labels):
    # set the model to training mode
    # because dropout has 2 different modes!
    model.train()

    inputs = Variable(inputs, requires_grad=False)
    labels = Variable(labels, requires_grad=False)

    # Reset gradient
    optimizer.zero_grad()

    # Forward
    logits = model.forward(inputs)
    output = loss.forward(logits, labels)

    # Backward
    output.backward()

    # Update parameters
    optimizer.step()

    # what's the difference between backward() and step()?

    return output.item()


# similar to train() but not doing the backprop step
def get_cost(model, loss, inputs, labels):
    # set the model to testing mode
    # because dropout has 2 different modes!
    model.eval()

    inputs = Variable(inputs, requires_grad=False)
    labels = Variable(labels, requires_grad=False)

    # Forward
    logits = model.forward(inputs)
    output = loss.forward(logits, labels)

    return output.item()


# define the prediction procedure
# also encapsulate these steps
# Note: inputs is a torch tensor
def predict(model, inputs):
    # set the model to testing mode
    # because dropout has 2 different modes!
    model.eval()

    inputs = Variable(inputs, requires_grad=False)
    logits = model.forward(inputs)
    return logits.data.numpy().argmax(axis=1)


# return the accuracy
# labels is a torch tensor
# to get back the internal numpy data
# use the instance method .numpy()
def score(model, inputs, labels):
    predictions = predict(model, inputs)
    return np.mean(labels.numpy() == predictions)


### prepare for training loop ###

# convert the data arrays into torch tensors
Xtrain = torch.from_numpy(Xtrain).float()
Ytrain = torch.from_numpy(Ytrain).long()
Xtest = torch.from_numpy(Xtest).float()
Ytest = torch.from_numpy(Ytest).long()

# training parameters
epochs = 15
batch_size = 32
n_batches = Xtrain.size()[0] // batch_size

# things to keep track of
train_costs = []
test_costs = []
train_accuracies = []
test_accuracies = []

# main training loop
for i in range(epochs):
    cost = 0
    test_cost = 0
    for j in range(n_batches):
        Xbatch = Xtrain[j * batch_size:(j + 1) * batch_size]
        Ybatch = Ytrain[j * batch_size:(j + 1) * batch_size]
        cost += train(model, loss, optimizer, Xbatch, Ybatch)

    # we could have also calculated the train cost here
    # but I wanted to show you that we could also return it
    # from the train function itself
    train_acc = score(model, Xtrain, Ytrain)
    test_acc = score(model, Xtest, Ytest)
    test_cost = get_cost(model, loss, Xtest, Ytest)

    print("Epoch: %d, cost: %f, acc: %.2f" % (i, test_cost, test_acc))

    # for plotting
    train_costs.append(cost / n_batches)
    train_accuracies.append(train_acc)
    test_costs.append(test_cost)
    test_accuracies.append(test_acc)

# plot the results
plt.plot(train_costs, label='Train cost')
plt.plot(test_costs, label='Test cost')
plt.title('Cost')
plt.legend()
plt.show()

plt.plot(train_accuracies, label='Train accuracy')
plt.plot(test_accuracies, label='Test accuracy')
plt.title('Accuracy')
plt.legend()
plt.show()

